In [2]:
import pandas as pd
from pprint import pprint
from astropy.table import Table, join
from astroquery.sdss import SDSS

In [3]:
# astroquery is weird - can query by region, plate, fiber id, or mjd
# query sdss objects by plate number ~ 500 objects per plate


# read in the csv file, and get objects on one plate
file = 'sdss_objects.csv'
df = pd.read_csv(file)
print(f"Total number of objects: {len(df)}")

plate_number = [445] 

# filter the data set by plate number
objects = df[df['plate'].isin(plate_number)]

print(f"Number of objects from plate: {len(objects)}")

# query sdss by plate numer, print available columns

results = SDSS.query_specobj(plate = 445)
print(results.columns)


# match the queries objects with the objects in the csv
objects = objects.copy()
query_df = results.to_pandas()

objects['specobjid'] = objects['specobjid'].astype('int64')
query_df['specobjid'] = query_df['specobjid'].astype('int64')

# merge the data frames to get the matched objects
matched_objects = pd.merge(objects, query_df, on='specobjid', how='inner')


obj_matches = Table.from_pandas(matched_objects)
obj_matches



Total number of objects: 860684
Number of objects from plate: 587
<TableColumns names=('ra','dec','objid','run','rerun','camcol','field','z','plate','mjd','fiberID','specobjid','run2d')>


specobjid,SOURCETYPE,RA,Dec,plate_x,ra,dec,objid,run,rerun,camcol,field,z,plate_y,mjd,fiberID,run2d
int64,str25,float64,float64,int64,float64,float64,uint64,int32,int32,int32,int32,float64,int32,int32,int32,int32
501025764846626816,GALAXY,130.61346,49.754185,445,130.61349960302,49.7542050007829,1237651189746565323,1331,301,1,175,0.1864123,445,51873,1,26
501026039724533760,GALAXY,130.50245,49.952646,445,130.502496814982,49.9526452500273,1237651249874534681,1345,301,1,151,0.3371834,445,51873,2,26
501026314602440704,GALAXY,130.80185,50.011891,445,130.801833171431,50.0118885755988,1237651249874600266,1345,301,1,152,0.186489,445,51873,3,26
501026589480347648,GALAXY,130.62652,50.037049,445,130.62652811744,50.037047131032,1237651249874600531,1345,301,1,152,0.4206361,445,51873,4,26
501026864358254592,GALAXY,130.66302,50.07183,445,130.663029522217,50.0718255938113,1237651249874600173,1345,301,1,152,0.09534973,445,51873,5,26
501027139236161536,GALAXY,130.65008,49.928713,445,130.650100893538,49.9287158883059,1237651249874534526,1345,301,1,151,0.1664238,445,51873,6,26
501027414114068480,QSO,130.55542,49.843956,445,130.555470999751,49.8439569299889,1237651249874534583,1345,301,1,151,2.030451,445,51873,7,26
501027688991975424,GALAXY,130.44962,49.955264,445,130.449688077166,49.9552790767358,1237651065191858693,1302,301,1,165,0.402256,445,51873,8,26
501027963869882368,GALAXY,130.54073,50.024759,445,130.540791289279,50.0247573027193,1237651190283501764,1331,301,2,176,0.18413,445,51873,9,26


In [4]:
# adjust columns such that dataframes match

if 'plate_y' in obj_matches.columns:
    obj_matches.remove_columns('plate_y')


if 'plate_x' in obj_matches.columns:
    obj_matches.rename_column('plate_x', 'plate')

# printt available columns
print("Columns in obj_matches:", obj_matches.columns)

# get spectra of matched objects
spec = SDSS.get_spectra(matches=obj_matches)


Columns in obj_matches: <TableColumns names=('specobjid','SOURCETYPE','RA','Dec','plate','ra','dec','objid','run','rerun','camcol','field','z','mjd','fiberID','run2d')>


In [6]:
# download data from spectra
spectra_table = []

for hdu_list in spec:
    # Extract the spectrum's binary table (usually in the 1st HDU)
    data = hdu_list[1].data
    header = hdu_list[0].header
    
    # get wavelength and flux data
    
    wavelength = data['loglam']
    flux = data['flux']


    # get information from header
    obj_id = header['SPEC_ID']      
    g_sn = header['SPEC1_G']
    r_sn = header['SPEC1_R']
    i_sn = header['SPEC1_I']
    
    red_shift = header['SHIFT']
    red_shift_err = header['SHIFTERR']
    
    central_wavelength = header['COEFF0']
    
    # append the data to a table
    spectra_table.append([obj_id, wavelength, flux, red_shift, red_shift_err, g_sn, r_sn, i_sn, central_wavelength])

# create an astropy table
spectra_table = Table(rows=spectra_table, names=['specobjid', 'wavelength', 'red shift', 'red shift error', 'flux', 'g sn', 'r sn', 'i sn', 'central wavelength'])

# add the spectra 
spectra_table['specobjid'] = spectra_table['specobjid'].astype('int64')
matched_with_sourcetype = join(obj_matches, spectra_table, keys='specobjid', join_type='left')

# save to csv
spectra_df = matched_with_sourcetype.to_pandas()
spectra_table.write('spectra_table.fits', overwrite=True)
spectra_df.to_csv('spectral_data2.csv', index=False)
print(len(spectra_df))

print(f"Rows in obj_matches: {len(obj_matches)}")



587
Rows in obj_matches: 587
